In [79]:
# Se importa la libreria random para hacer uso de números aleatorios, y una libreria para guardar listas
import pygame
import math
import random
import heapq
from collections import defaultdict, deque

In [80]:
class Nodo: # Clase nodo guarda el nombre de cada nodo en forma de string (1)
    def __init__(self, valor):
        self.valor = valor
        self.vecinos = []
        self.vecinos2 = []
    # Se crea un metodo para agregar vecinos al arbol generado utilizado en BFS y DFS
    def agregar_vecino(self, vecino):
        self.vecinos.append(vecino)
    def agregar_vecino2(self, vecino, peso):
        self.vecinos2.append((vecino,peso))
    # El nodo se guarda con el formato dado desde la función
    def __str__(self):
        return str(self.valor)

class Arista: # Clase arista guarda el nombre de cada arista en forma de string ( 1 -> 2 )
    def __init__(self, origen, destino, peso):
        self.origen = origen
        self.destino = destino
        self.origen2 = origen
        self.destino2 = destino
        self.peso = peso
    # La arista se guarda con el formato 1 -> 2
    def __str__(self):
        return f"{self.origen} -> {self.destino}"

class Grafo:
    def __init__(self,nombre_archivo):
        # Se inicializa el arreglo de nodos y aristas del grafo
        self.nodos = {}
        self.aristas = {}
        self.pesos = {}
        self.aristas2 = {}
        self.pesos2 = {}
        self.aristas3 = []
        self.nombre_archivo = nombre_archivo
    
    # Se guarda cada nodo a una lista de nodos
    def agregar_nodo(self, valor):
        if valor not in self.nodos:
            nodo = Nodo(valor)
            self.nodos[valor] = nodo
    # Se guarda cada arista a una lista de aristas
    def agregar_arista(self, valor_origen, valor_destino, peso):
        #if valor_origen not in self.nodos and valor_destino not in self.nodos:
        # Se verifica que el nodo de origen y destino esten en la lista de nodos
        if valor_origen not in self.nodos:
            self.agregar_nodo(valor_origen)
        if valor_destino not in self.nodos:
            self.agregar_nodo(valor_destino)

        origen = self.nodos[valor_origen]
        destino = self.nodos[valor_destino]
        origen2 = self.nodos[valor_origen]
        destino2 = self.nodos[valor_destino]
        # Se agrega el nodo vecino para BFS y DFS
        origen.agregar_vecino(destino)
        
        origen2.agregar_vecino2(destino2, peso)
        destino2.agregar_vecino2(origen2, peso)

        arista = Arista(valor_origen, valor_destino, peso)
        self.aristas2[(valor_origen, valor_destino)] = arista
        self.pesos2[(valor_origen, valor_destino)] = peso
        
        self.aristas3.append(arista)
        self.pesos2[(valor_origen, valor_destino)] = peso
        self.pesos2[(valor_destino, valor_origen)] = peso
        
        arista = Arista(valor_origen, valor_destino, peso)
        if valor_origen not in self.aristas:
            self.aristas[valor_origen] = []
        self.aristas[valor_origen].append((arista, peso))
        
    def __str__(self):
        """    
        # Se guardan los valores de los nodos y aristas con el formato:
        digraph G {
            // Nodos
            1;
            2;
            3;
            // Aristas
            1 -> 2;
            1 -> 3;
            2 -> 3;
        }
        """
        output = "Grafo generado\ndigraph G { \n\t// Nodos:\n"
        for nodo in self.nodos.values():
            output += "\t" + str(nodo.valor) + "\n"

        output += "\t// Aristas:\n"
        for origen, aristas in self.aristas.items():
            for arista, peso in aristas:
                #output += "({}) --{}--> ({})\n".format(origen, peso, arista.destino)
                output += "\t{} -> {} [label=\"{}\"];\n".format(origen, arista.destino, peso)
        # Se guarda la cadena de texto como un archivo gv
        with open(self.nombre_archivo, "w") as archivo:
            archivo.write(output+"}")
        return output+"}"    
        
    # Se define el metodo para el algoritmo BFS
    def bfs(self, valor_inicio):
        # Se verifica que el nodo de inicio se encuentre en el grafo
        if valor_inicio not in self.nodos:
            return
        visitado = set() # Se guardan los nodos visitados en el conjunto generado
        cola = deque([self.nodos[valor_inicio]]) # Se define una "cola" con los nodos visitados
        padres = {self.nodos[valor_inicio]: None} # Se guardan los nodos padres de cada arista para obtener el arbol BFS
        while cola: # Se establece la condicion de paro del algoritmo cando no hay mas elementos en la cola
            nodo_actual = cola.popleft()
            if nodo_actual not in visitado: # Se establece si el nodo actual no se ha visitado
                #print(nodo_actual.valor, end=" ")
                visitado.add(nodo_actual) # Se marca el nodo como visitado
                for vecino in nodo_actual.vecinos:
                    if vecino not in visitado:
                        cola.append(vecino) # Se agrega el nodo vecino a la cola
                        padres[vecino] = nodo_actual # Se guarda el nodo de origen o nodo padre para cada arista
        arbol_bfs = {} # Se inicializa un diccionario con los elementos del arbol generado por BFS
        for nodo, padre in padres.items():
            if padre is not None:
                # Se establecen los nodos y aristas del arbol generado
                arbol_bfs.setdefault(padre.valor, []).append(nodo.valor)
        return arbol_bfs
    
    # Se define el metodo para el algoritmo DFS
    def dfs(self, valor_inicio):
        # Se verifica que el nodo de inicio se encuentre en el grafo
        if valor_inicio not in self.nodos:
            return
        visitado = set() # Se guardan los nodos visitados en el conjunto generado
        padres = {self.nodos[valor_inicio]: None}  # Se guardan los nodos padres de cada arista para obtener el arbol 
        def dfs_recursivo(nodo_actual): # Se define una funcion con el algoritmo de forma iterativa
            #print(nodo_actual.valor, end=" ")
            visitado.add(nodo_actual) # Se marca el nodo como visitado
            for vecino in nodo_actual.vecinos:
                if vecino not in visitado:
                    dfs_recursivo(vecino)
                    padres[vecino] = nodo_actual # Se guarda el nodo de origen o nodo padre para cada arista
        dfs_recursivo(self.nodos[valor_inicio]) # Se llama al algoritmo iterativo
        arbol_dfs = {} # Se inicializa un diccionario con los elementos del arbol generado por BFS
        for hijo, padre in padres.items():
            if padre is not None:
                # Se establecen los nodos y aristas del arbol generado
                arbol_dfs.setdefault(padre, []).append(hijo)
        return arbol_dfs
    
    def Dijkstra(self, valor_inicio):
        # Se inicializan todas las distancias a todos los nodos en infinito (nodos desconectados)
        distancias = {valor: float('inf') for valor in self.nodos}
        distancias[valor_inicio] = 0 # Se inicializa en cero la distancia desde el primer nodo
        predecesor = {valor: None for valor in self.nodos} # Se obtiene el siguiente nodo conectado al nodo actual
        visitados = set()
        while len(visitados) < len(self.nodos): # Se itera hasta que se recorren todos los nodos
            nodo_actual = None
            distancia_minima = float('inf')
            for valor, distancia in distancias.items():
                if valor not in visitados and distancia < distancia_minima:
                    nodo_actual = valor # Se establece el nodo actual
                    distancia_minima = distancia # Se establece la distancia al nodo actual
            if nodo_actual is None:
                break
            visitados.add(nodo_actual) # Se marcan los nodos visitados
            if nodo_actual in self.aristas:
                for arista, peso in self.aristas[nodo_actual]:
                    nueva_distancia = distancias[nodo_actual] + peso # Se acumulan los pesos
                    if nueva_distancia < distancias[arista.destino]:
                        distancias[arista.destino] = nueva_distancia
                        predecesor[arista.destino] = nodo_actual
        return distancias, predecesor
    
    def find(self, parent, i):
        if parent[i] == i:
            return i
        else:
            return self.find(parent, parent[i])

    def union(self, parent, rank, x, y):
        xroot = self.find(parent, x)
        yroot = self.find(parent, y)
        if rank[xroot] < rank[yroot]:
            parent[xroot] = yroot
        elif rank[xroot] > rank[yroot]:
            parent[yroot] = xroot
        else:
            parent[yroot] = xroot
            rank[xroot] += 1
            
    def is_connected(self):
        # Verificar si el grafo está conectado utilizando BFS o DFS
        if not self.nodos:
            return True
        start = next(iter(self.nodos))
        visited = set()
        stack = [start]
        while stack:
            node = stack.pop()
            if node not in visited:
                visited.add(node)
                for vecino in self.nodos[node].vecinos:
                    if vecino.valor not in visited:
                        stack.append(vecino.valor)
        return len(visited) == len(self.nodos)
            
    def kruskalD(self):
        # Lista para almacenar las aristas del MST
        mst = []
        # Ordenar las aristas por peso
        aristas_ordenadas = sorted(self.pesos2.items(), key=lambda item: item[1])
        parent = {}
        rank = {}
        # Inicializar los conjuntos disjuntos
        for nodo in self.nodos:
            parent[nodo] = nodo
            rank[nodo] = 0
        for (origen, destino), peso in aristas_ordenadas:
            u = origen
            v = destino
            uroot = self.find(parent, u)
            vroot = self.find(parent, v)
            if uroot != vroot:
                mst.append((self.aristas2[(origen, destino)], peso))
                self.union(parent, rank, uroot, vroot)
        return mst
    
    def kruskalI(self):
        # Ordenar las aristas por peso en orden descendente
        aristas_ordenadas = sorted(self.pesos2.items(), key=lambda item: item[1], reverse=True)
        # Eliminar aristas en orden descendente de peso, si no desconectan el grafo
        for (origen, destino), peso in aristas_ordenadas:
            # Eliminar temporalmente la arista
            self.nodos[origen].vecinos = [vecino for vecino in self.nodos[origen].vecinos if vecino.valor != destino]
            if not self.is_connected():
                # Si el grafo se desconecta, volver a añadir la arista
                self.nodos[origen].agregar_vecino(self.nodos[destino])
            else:
                # Eliminar la arista permanentemente
                del self.aristas2[(origen, destino)]
                del self.pesos2[(origen, destino)]
        # El MST estará compuesto por las aristas restantes en el grafo
        mst = [(arista, peso) for (origen, destino), arista in self.aristas2.items() for (key, peso) in self.pesos2.items() if key == (origen, destino)]
        return mst
    
    def prim(self, inicio):
        # Verificar si el nodo inicial existe en el grafo
        if inicio not in self.nodos:
            return None
        # Lista para almacenar las aristas del MST
        mst = []
        # Cola de prioridades para seleccionar la arista de menor peso
        aristas_heap = []
        # Conjunto de nodos visitados
        visitados = set()
        # Función para agregar las aristas de un nodo a la cola de prioridades
        def agregar_aristas(nodo):
            for vecino, peso in self.nodos[nodo].vecinos2:
                if vecino.valor not in visitados:
                    heapq.heappush(aristas_heap, (peso, nodo, vecino.valor))
        # Comenzar desde el nodo inicial
        visitados.add(inicio)
        agregar_aristas(inicio)
        while aristas_heap:
            peso, origen, destino = heapq.heappop(aristas_heap)
            if destino not in visitados:
                visitados.add(destino)
                mst.append((origen, destino, peso))
                agregar_aristas(destino)
        return mst

    def to_adjacency_list(self):
        adj_list = {}
        for nodo in self.nodos.values():
            if nodo.valor not in adj_list:
                adj_list[nodo.valor] = []
            for vecino in nodo.vecinos:
                adj_list[nodo.valor].append(vecino.valor)
                if vecino.valor not in adj_list:
                    adj_list[vecino.valor] = []
                if nodo.valor not in adj_list[vecino.valor]:
                    adj_list[vecino.valor].append(nodo.valor)
        return adj_list

In [81]:
def escribir_Dijkstra(grafo,n,nombre_archivo):
    print("Arbol de pesos minimos desde el nodo " + str(n) + "\ndigraph G_nodo_origen_" + str(n) + " {")
    distancias, predecesor = grafo.Dijkstra(n) # Se obtienen los nodos conectados al origen n, y su distancia
    with open(nombre_archivo, "w") as archivo: # Se guarda el grafo en un archivo gv
        archivo.write("digraph G_nodo_origen_" + str(n) + " {\n")
        for nodo, distancia in distancias.items():
            if distancia == float('inf') or distancia == 0: # Se eliminan los nodos desconectados del arbol
                pass
            else:
                #print("Distancia desde el nodo {} hasta {}: {}".format(n, nodo, distancia))
                archivo.write("\t{} -> nodo_{}({});\n".format(n, nodo, distancia))
                print("\t{} -> nodo_{}({});".format(n, nodo, distancia))
        archivo.write("}")
    print("}")

In [82]:
def escribir_KruskalD(grafo,nombre_archivo):
    print("Arbol de expansion minima Kruskal directo \ndigraph G {")
    mst = grafo.kruskalD()
    with open(nombre_archivo, "w") as archivo: # Se guarda el grafo en un archivo gv
        archivo.write("digraph G {\n")
        for arista, peso in mst:
            archivo.write("\t{} -> {} [label=\"{}\"];\n".format(arista.origen, arista.destino, peso))
            print("\t{} -> {} [label=\"{}\"];".format(arista.origen, arista.destino, peso))
        archivo.write("}")
    print("}")

In [83]:
def escribir_KruskalI(grafo,nombre_archivo):
    print("Arbol de expansion minima Kruskal inverso \ndigraph G {")
    mst = grafo.kruskalD()
    with open(nombre_archivo, "w") as archivo: # Se guarda el grafo en un archivo gv
        archivo.write("digraph G {\n")
        for arista, peso in mst:
            archivo.write("\t{} -> {} [label=\"{}\"];\n".format(arista.origen, arista.destino, peso))
            print("\t{} -> {} [label=\"{}\"];".format(arista.origen, arista.destino, peso))
        archivo.write("}")
    print("}")

In [84]:
def escribir_Prim(grafo,nombre_archivo):
    print("Arbol de expansion minima Prim \ndigraph G {")
    mst = grafo.prim(1)
    with open(nombre_archivo, "w") as archivo: # Se guarda el grafo en un archivo gv
        archivo.write("digraph G {\n")
        for origen, destino, peso in mst:
            archivo.write("\t{} -> {} [label=\"{}\"];\n".format(origen, destino, peso))
            print("\t{} -> {} [label=\"{}\"];".format(origen, destino, peso))
        archivo.write("}")
    print("}")

In [85]:
def randomErdos(n,m,p,w):
    nombre_archivo = "Erdos_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    # Se agregan n nodos al grafo
    for i in range(n):
        g.agregar_nodo((i))
    # se conectan m nodos de forma aleatoria. (n>m)
    for i in range(m):
        u = random.randint(0,n-1)
        v = random.randint(0,n-1)
        if u != v: # Se omiten las conexiones hacia un mismo nodo
            g.agregar_arista((u),(v),random.randint(1,w))
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(p)
    arbol_dfs = g.dfs(p)
    #escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    #escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    #escribir_Dijkstra(g,p,"Dijkstra_" + nombre_archivo)
    #escribir_KruskalD(g,"KruskalD_" + nombre_archivo)
    #escribir_KruskalI(g,"KruskalI_" + nombre_archivo)
    #escribir_Prim(g,"Prim_" + nombre_archivo)
    return g

In [86]:
def Malla(n,m,p,w):
    nombre_archivo = "Malla_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    # Se agregan n*m nodos al grafo
    for i in range(n*m):
        g.agregar_nodo((i+1))
    # Se conectan los nodos de la malla i,j+1 y i+1,j
    for i in range(n):
        for j in range(m):
            # La malla se recorre de la siguiente forma:
            """
            1  2  3
            4  5  6
            7  8  9
            """
            u = i*n+j+1
            v1 = i*n+j+2
            v2 = (i+1)*n+j+1
            # Se descartan todas las posibles combinaciones cuyos nodos superen el valor total de nodos en el grafo
            if v1 > n*m or v2 > n*m:
                v2 = u
            if u != v1: # Se omiten las conexiones hacia un mismo nodo
                g.agregar_arista((u),(v1),random.randint(1,w))
            if u != v2: # Se omiten las conexiones hacia un mismo nodo
                g.agregar_arista((u),(v2),random.randint(1,w))
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(p)
    arbol_dfs = g.dfs(p)
    #escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    #escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    #escribir_Dijkstra(g,p,"Dijkstra_" + nombre_archivo)
    #escribir_KruskalD(g,"KruskalD_" + nombre_archivo)
    #escribir_KruskalI(g,"KruskalI_" + nombre_archivo)
    #escribir_Prim(g,"Prim_" + nombre_archivo)
    return g

In [87]:
def Geografico(n,r,p,w):
    nombre_archivo = "Geografico_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    x = []; y = []
    # Se agregan n nodos al grafo
    for i in range(n):
        g.agregar_nodo((i))
        # se define una posición aleatria del nodo en el espacio
        x.append(random.randint(0,3*r))
        y.append(random.randint(0,3*r))
    for i in range(n):
        for j in range(n):
            dist = (x[i]-x[j])**2+(y[i]-y[j])**2 # Se calcula la distancia entre los nodos del grafo
            # Se conectan los nodos que se encuentran dentro del rango de distancia dado
            if dist < r and i != j: # Se omiten las conexiones hacia un mismo nodo
                g.agregar_arista((i),(j),random.randint(1,w))
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(p)
    arbol_dfs = g.dfs(p)
    #escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    #escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    #escribir_Dijkstra(g,p,"Dijkstra_" + nombre_archivo)
    #escribir_KruskalD(g,"KruskalD_" + nombre_archivo)
    #escribir_KruskalI(g,"KruskalI_" + nombre_archivo)
    #escribir_Prim(g,"Prim_" + nombre_archivo)
    return g

In [88]:
def Gilbert(n,p,x,w):
    nombre_archivo = "Gilbert_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    # Se agregan n nodos al grafo
    for i in range(n):
        g.agregar_nodo((i))
    for i in range(n):
        lista = [] # Se guardan los nodos previamente conectados para evitar que se dupliquen las aristas
        for j in range(n):
            # Se conectan 2 nodos de forma aleatoria con una probabilidad p
            cond = j in lista # Se determina si el nodo se encuentra en el bucle actual para evitar repeticiones
            if random.random() <= p and i != j and cond == False: # Se omiten las conexiones hacia un mismo nodo
                g.agregar_arista((i),(j),random.randint(1,w))
            lista.append(j) # Se guardan los nodos actualmente conectados para evitar que se dupliquen las aristas
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(x)
    arbol_dfs = g.dfs(x)
    #escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    #escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    #escribir_Dijkstra(g,x,"Dijkstra_" + nombre_archivo)
    #escribir_KruskalD(g,"KruskalD_" + nombre_archivo)
    #escribir_KruskalI(g,"KruskalI_" + nombre_archivo)
    #escribir_Prim(g,"Prim_" + nombre_archivo)
    return g

In [89]:
def Barabasi(n,m,p,w):
    nombre_archivo = "Barabasi_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    # Se agregan m nodos al grafo
    for i in range(1,m):
        g.agregar_nodo((i))
        # Se conectan los m nodos agregados
        for j in range(1,i):
            g.agregar_arista((i),(j),random.randint(1,w))
    # Se determina la preferencia de conexión de los nodos añadidos
    preferencia = [nodo for nodo in g.nodos.keys() for _ in range(len(g.nodos))]
    for i in range(m, n):
        lista = [] # Se guardan los nodos previamente conectados para evitar que se dupliquen las aristas
        g.agregar_nodo(i) # Se agrega un nuevo nodo
        conexiones = random.sample(preferencia, m) # Se conecta el nuevo nodo al grafo
        for nodo in conexiones:
            cond = nodo in lista # Se determina si el nodo se encuentra en el bucle actual para evitar repeticiones
            if i != nodo and cond == False: # Se omiten las conexiones hacia un mismo nodo y las aristas repetidas
                g.agregar_arista(i, nodo, random.randint(1,w))
                preferencia.extend([nodo, i])
            lista.append(nodo) # Se guardan los nodos actualmente conectados para evitar que se dupliquen las aristas
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(p)
    arbol_dfs = g.dfs(p)
    #escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    #escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    #escribir_Dijkstra(g,p,"Dijkstra_" + nombre_archivo)
    #escribir_KruskalD(g,"KruskalD_" + nombre_archivo)
    #escribir_KruskalI(g,"KruskalI_" + nombre_archivo)
    #escribir_Prim(g,"Prim_" + nombre_archivo)
    return g

In [90]:
def Dorogovtsev(n,p,w):
    nombre_archivo = "Dorogovtsev_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    # Se agregan 3 nodos al grafo
    for i in range(3):
        g.agregar_nodo(i)
    # Se conectan los nodos agregados formando un triángulo
    g.agregar_arista(1, 2, random.randint(1,w))
    g.agregar_arista(2, 3, random.randint(1,w))
    g.agregar_arista(3, 1, random.randint(1,w))
    # Se agregan los n-3 nodos restantes
    for i in range(4, n):
        lista = [] # Se guardan los nodos previamente conectados para evitar que se dupliquen las aristas
        g.agregar_nodo((i)) # Se agrega un nuevo nodo
        for k in range(4):
            j = random.choice(list(g.nodos.keys())) # Se escoge el nodo destino de forma aleatoria de las aristas existentes
            cond = j in lista # Se determina si el nodo se encuentra en el bucle actual para evitar repeticiones
            if i != j and cond == False: # Se omiten las conexiones hacia un mismo nodo y las aristas repetidas
                g.agregar_arista((i), (j), random.randint(1,w))
            lista.append(j) # Se guardan los nodos actualmente conectados para evitar que se dupliquen las aristas
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(p)
    arbol_dfs = g.dfs(p)
    #escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    #escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    #escribir_Dijkstra(g,p,"Dijkstra_" + nombre_archivo)
    #escribir_KruskalD(g,"KruskalD_" + nombre_archivo)
    #escribir_KruskalI(g,"KruskalI_" + nombre_archivo)
    #escribir_Prim(g,"Prim_" + nombre_archivo)
    return g

In [91]:
# Definir la función de distancia y el algoritmo de disposición por resortes
def distance(p1, p2):
    """Calcula la distancia euclidiana entre dos puntos"""
    return 0.09*math.sqrt((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2)

In [100]:
def spring_layout(graph, positions, k, C, t, max_iterations, win):
    """Algoritmo de disposición de resortes"""
    WIDTH, HEIGHT = 1920, 1080
    for iteration in range(max_iterations):
        forces = {node: [0, 0] for node in graph}
        
        # Calcular fuerzas de repulsión
        for v in graph:
            for u in graph:
                if u != v:
                    delta = (positions[v][0] - positions[u][0], positions[v][1] - positions[u][1])
                    dist = distance(positions[v], positions[u])
                    if dist > 0:
                        repulsive_force = k ** 2 / dist
                        forces[v][0] += delta[0] / dist * repulsive_force
                        forces[v][1] += delta[1] / dist * repulsive_force
        
        # Calcular fuerzas de atracción
        for v in graph:
            for u in graph[v]:
                delta = (positions[v][0] - positions[u][0], positions[v][1] - positions[u][1])
                dist = distance(positions[v], positions[u])
                if dist > 0:
                    attractive_force = (dist ** 2) / k
                    forces[v][0] -= delta[0] / dist * attractive_force
                    forces[v][1] -= delta[1] / dist * attractive_force

        # Actualizar posiciones
        for v in graph:
            positions[v] = (
                positions[v][0] + forces[v][0] * t,
                positions[v][1] + forces[v][1] * t
            )

            # Limitar las posiciones dentro de la ventana
            positions[v] = (
                min(max(positions[v][0], 50), WIDTH - 50),
                min(max(positions[v][1], 50), HEIGHT - 50)
            )

        # Dibujar el grafo en cada iteración
        draw_graph(win, graph, positions)
        pygame.time.delay(50)

    return positions

In [93]:
# Función para dibujar el grafo
def draw_graph(win, graph, positions):
    WHITE = (255, 255, 255)
    BLACK = (0, 0, 0)
    RED = (255, 0, 0)
    win.fill(WHITE)

    # Dibujar aristas
    for node in graph:
        for neighbor in graph[node]:
            pygame.draw.line(win, BLACK, positions[node], positions[neighbor], 1)
    
    # Dibujar nodos
    for node, pos in positions.items():
        pygame.draw.circle(win, RED, (int(pos[0]), int(pos[1])), 5)
    
    pygame.display.update()

In [98]:
def Grafo_dibujo(grafo,k,C,t,max_iterations):
    # Configuración de Pygame
    pygame.init()
    WIDTH, HEIGHT = 1920, 1080
    win = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("Spring Layout Graph Visualization")

    # Colores
    WHITE = (255, 255, 255)
    BLACK = (0, 0, 0)
    RED = (255, 0, 0)

    # Convertir el grafo a una lista de adyacencia
    graph = grafo.to_adjacency_list()
    print(graph)
    print(grafo)

    # Inicializar posiciones de los nodos aleatoriamente dentro de los límites de la ventana
    positions = {node: (random.uniform(50, WIDTH - 50), random.uniform(50, HEIGHT - 50)) for node in graph}
    # Escalar posiciones iniciales para asegurar que se ajustan al área de dibujo
    positions = scale_positions(positions, 0.7)

    # Calcular la disposición usando el algoritmo de resortes
    positions = spring_layout(graph, positions, k, C, t, max_iterations,win)
        
    # Bucle principal
    running = True
    #iteration = 0
    while running :#and iteration < max_iterations:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
    
        positions = spring_layout(graph, positions, k, C, t, 1, win)
        #iteration += 1
    #entrada = input('presiona enter')
    pygame.quit()

In [95]:
# Función para escalar posiciones
def scale_positions(positions, scale_factor):
    for node in positions:
        positions[node] = (
            positions[node][0] * scale_factor,
            positions[node][1] * scale_factor
        )
    return positions

In [96]:
# Crear el grafo y agregar nodos y aristas
grafo1 = randomErdos(100,100,15,25)
grafo2 = randomErdos(350,350,15,25)
grafo3 = Malla(10,10,50,25)
grafo4 = Malla(17,20,15,25)
grafo5 = Geografico(100,10,20,25)
grafo6 = Geografico(350,10,50,25)
grafo7 = Gilbert(100,0.1,12,50)
grafo8 = Gilbert(350,0.01,110,75)
grafo9 = Dorogovtsev(100,14,25)
grafo10 = Dorogovtsev(350,200,25)
grafo11 = Barabasi(100,10,20,25)
grafo12 = Barabasi(350,10,150,25)

In [102]:
# Erdos 100 nodos
# Parámetros
k = 1.7  # Constante de repulsión
C = 0.005  # Constante de atracción
t = 0.001  # Factor de tiempo 
max_iterations = 100  # Número máximo de iteraciones
Grafo_dibujo(grafo1,k,C,t,max_iterations)

{0: [40], 1: [64, 99], 64: [1], 2: [28, 47, 55], 3: [], 4: [11, 32, 36], 11: [4, 47], 5: [86], 86: [5, 8, 41, 90], 6: [53, 32, 83], 53: [6, 54, 93, 95], 7: [35], 35: [7], 8: [86, 84, 42, 70], 84: [8, 89, 42], 9: [91, 58], 91: [9], 10: [], 47: [11, 95, 2], 12: [54, 29, 41, 62], 54: [12, 23, 36, 51, 53], 29: [12], 13: [43, 63], 14: [36, 38], 36: [14, 83, 4, 54], 38: [14], 15: [43, 42, 71, 92], 43: [15, 13, 88], 16: [33, 52, 82], 33: [16, 93, 44], 52: [16, 99], 82: [16, 20, 30, 57, 65], 17: [], 18: [89, 22, 41], 89: [18, 84, 24], 22: [18], 19: [26], 26: [19, 68], 20: [82, 41, 55], 21: [], 23: [97, 54], 97: [23, 68], 24: [51, 55, 89], 25: [76, 37], 76: [25, 66], 37: [25, 28], 68: [26, 32, 87, 97], 27: [94], 28: [79, 2, 37], 79: [28, 60, 95, 81, 58, 98], 30: [82], 31: [], 32: [68, 6, 4, 95], 93: [33, 53], 34: [50], 50: [34, 44, 74, 75], 83: [36, 6], 39: [96], 40: [0], 41: [86, 46, 12, 18, 20], 46: [41], 42: [94, 8, 15, 84], 94: [42, 72, 27, 72, 75], 44: [50, 33], 45: [], 95: [47, 79, 53, 32

In [103]:
# Erdos 350 nodos
# Parámetros
k = 1.7  # Constante de repulsión
C = 0.005  # Constante de atracción
t = 0.001  # Factor de tiempo
max_iterations = 100  # Número máximo de iteraciones
Grafo_dibujo(grafo2,k,C,t,max_iterations)

{0: [153], 153: [0, 89, 248, 135, 0, 317], 1: [306, 319], 2: [69, 86], 69: [2, 328], 3: [251, 213, 222], 251: [3, 18], 4: [], 5: [199], 6: [283, 235, 145, 338], 283: [6, 91, 318], 235: [6, 81], 7: [], 8: [159, 98, 311], 159: [8, 311, 199, 210, 293, 216, 348], 9: [233], 10: [224], 11: [], 12: [156], 156: [12], 13: [83, 85, 165], 14: [98, 101], 98: [14, 83, 8], 101: [14, 111, 202, 300], 15: [], 16: [230], 230: [16, 106, 77, 323, 161], 17: [330, 225, 246], 330: [17, 277, 103], 225: [17, 271], 246: [17, 162, 287, 111], 18: [251, 42, 84], 19: [199, 349, 228], 199: [19, 73, 159, 61, 93, 317, 5], 349: [19, 287], 20: [59, 74], 59: [20, 171, 83, 245], 74: [20], 21: [38, 162], 22: [168], 23: [344], 24: [221], 221: [24, 109, 131, 229], 25: [135, 86, 72, 313], 135: [25, 67, 136, 137, 153, 313], 86: [25, 73, 77, 2, 303, 57, 88], 26: [29, 243, 70], 29: [26, 139, 115], 243: [26, 75], 27: [280, 167], 280: [27, 38, 60, 71], 28: [220], 220: [28], 139: [29, 297, 188, 231], 30: [303, 120, 126, 173], 303: 

In [109]:
# Malla 100 nodos
# Parámetros
k = 1.7  # Constante de repulsión
C = 0.005  # Constante de atracción
t = 0.005  # Factor de tiempo
max_iterations = 100  # Número máximo de iteraciones
Grafo_dibujo(grafo3,k,C,t,max_iterations)

{1: [2, 11], 2: [1, 3, 12], 11: [1, 10, 12, 21], 3: [2, 4, 13], 12: [2, 11, 13, 22], 4: [3, 5, 14], 13: [3, 12, 14, 23], 5: [4, 6, 15], 14: [4, 13, 15, 24], 6: [5, 7, 16], 15: [5, 14, 16, 25], 7: [6, 8, 17], 16: [6, 15, 17, 26], 8: [7, 9, 18], 17: [7, 16, 18, 27], 9: [8, 10, 19], 18: [8, 17, 19, 28], 10: [9, 11, 20], 19: [9, 18, 20, 29], 20: [10, 19, 21, 30], 21: [11, 20, 22, 31], 22: [12, 21, 23, 32], 23: [13, 22, 24, 33], 24: [14, 23, 25, 34], 25: [15, 24, 26, 35], 26: [16, 25, 27, 36], 27: [17, 26, 28, 37], 28: [18, 27, 29, 38], 29: [19, 28, 30, 39], 30: [20, 29, 31, 40], 31: [21, 30, 32, 41], 32: [22, 31, 33, 42], 33: [23, 32, 34, 43], 34: [24, 33, 35, 44], 35: [25, 34, 36, 45], 36: [26, 35, 37, 46], 37: [27, 36, 38, 47], 38: [28, 37, 39, 48], 39: [29, 38, 40, 49], 40: [30, 39, 41, 50], 41: [31, 40, 42, 51], 42: [32, 41, 43, 52], 43: [33, 42, 44, 53], 44: [34, 43, 45, 54], 45: [35, 44, 46, 55], 46: [36, 45, 47, 56], 47: [37, 46, 48, 57], 48: [38, 47, 49, 58], 49: [39, 48, 50, 59], 

In [110]:
# Malla 350 nodos
# Parámetros
k = 1.7  # Constante de repulsión
C = 0.005  # Constante de atracción
t = 0.003  # Factor de tiempo
max_iterations = 100  # Número máximo de iteraciones
Grafo_dibujo(grafo4,k,C,t,max_iterations)

{1: [2, 18], 2: [1, 3, 19], 18: [1, 17, 19, 35, 19, 35], 3: [2, 4, 20], 19: [2, 18, 20, 36, 20, 36], 4: [3, 5, 21], 20: [3, 19, 21, 37, 21, 37], 5: [4, 6, 22], 21: [4, 20, 22, 38], 6: [5, 7, 23], 22: [5, 21, 23, 39], 7: [6, 8, 24], 23: [6, 22, 24, 40], 8: [7, 9, 25], 24: [7, 23, 25, 41], 9: [8, 10, 26], 25: [8, 24, 26, 42], 10: [9, 11, 27], 26: [9, 25, 27, 43], 11: [10, 12, 28], 27: [10, 26, 28, 44], 12: [11, 13, 29], 28: [11, 27, 29, 45], 13: [12, 14, 30], 29: [12, 28, 30, 46], 14: [13, 15, 31], 30: [13, 29, 31, 47], 15: [14, 16, 32], 31: [14, 30, 32, 48], 16: [15, 17, 33], 32: [15, 31, 33, 49], 17: [16, 18, 34], 33: [16, 32, 34, 50], 34: [17, 33, 35, 51], 35: [18, 34, 36, 52, 36, 52], 36: [19, 35, 37, 53, 37, 53], 37: [20, 36, 38, 54, 38, 54], 38: [21, 37, 39, 55], 39: [22, 38, 40, 56], 40: [23, 39, 41, 57], 41: [24, 40, 42, 58], 42: [25, 41, 43, 59], 43: [26, 42, 44, 60], 44: [27, 43, 45, 61], 45: [28, 44, 46, 62], 46: [29, 45, 47, 63], 47: [30, 46, 48, 64], 48: [31, 47, 49, 65], 49

In [111]:
# Geografico 100 nodos
# Parámetros
k = 1.7  # Constante de repulsión
C = 0.005  # Constante de atracción
t = 0.005  # Factor de tiempo
max_iterations = 100  # Número máximo de iteraciones
Grafo_dibujo(grafo5,k,C,t,max_iterations)

{0: [6], 6: [0, 0, 66], 1: [4, 5, 60, 97], 4: [1, 1, 5, 60, 97], 5: [1, 4, 1, 4, 60, 97], 60: [1, 4, 5, 1, 4, 5], 97: [1, 4, 5, 80, 1, 4, 5, 80], 2: [14, 41, 77], 14: [2, 2, 33, 41, 67], 41: [2, 14, 2, 14, 77], 77: [2, 41, 2, 41], 3: [18, 20, 79], 18: [3, 3, 34, 79], 20: [3, 3, 23, 30, 79, 81], 79: [3, 18, 20, 3, 18, 20], 66: [6, 6, 75], 7: [31, 52, 78], 31: [7, 7, 52, 61, 78], 52: [7, 31, 7, 31, 61], 78: [7, 31, 53, 65, 7, 31, 53, 65, 93], 8: [9, 19, 76], 9: [8, 8, 19], 19: [8, 9, 8, 9, 76], 76: [8, 19, 21, 8, 19, 21], 10: [22], 22: [10, 10, 56], 11: [], 12: [13, 69], 13: [12, 12, 47], 69: [12, 12], 47: [13, 36, 13, 36], 33: [14, 14, 40, 67, 91], 67: [14, 33, 14, 33, 91], 15: [89, 96], 89: [15, 83, 15, 83, 96], 96: [15, 83, 89, 15, 83, 89], 16: [54, 95], 54: [16, 16, 95], 95: [16, 54, 16, 54], 17: [49], 49: [17, 26, 17, 26, 63], 34: [18, 18, 87], 23: [20, 20, 30, 81], 30: [20, 23, 20, 23, 51, 81], 81: [20, 23, 30, 51, 20, 23, 30, 51], 21: [58, 76], 58: [21, 56, 21, 56], 56: [22, 22, 5

In [115]:
# Geografico 350 nodos
# Parámetros
k = 1.7  # Constante de repulsión
C = 0.005  # Constante de atracción
t = 0.0015  # Factor de tiempo
max_iterations = 100  # Número máximo de iteraciones
Grafo_dibujo(grafo6,k,C,t,max_iterations)

{0: [139, 178, 188, 216, 221, 225, 253, 303, 332], 139: [0, 6, 59, 0, 6, 59, 178, 188, 192, 216, 221, 225, 229, 284, 303, 311], 178: [0, 6, 57, 59, 87, 101, 106, 139, 0, 6, 57, 59, 87, 101, 106, 139, 187, 188, 192, 216, 225, 229, 284, 295, 308, 309, 311], 188: [0, 6, 59, 139, 178, 0, 6, 59, 139, 178, 192, 216, 221, 225, 229, 284, 303, 311], 216: [0, 6, 139, 178, 188, 192, 0, 6, 139, 178, 188, 192, 221, 225, 229, 284, 303, 311], 221: [0, 6, 12, 139, 188, 192, 193, 216, 0, 6, 12, 139, 188, 192, 193, 216, 225, 258, 273, 303, 313], 225: [0, 6, 12, 139, 178, 188, 192, 216, 221, 0, 6, 12, 139, 178, 188, 192, 216, 221, 229, 273, 303, 311, 332], 253: [0, 25, 71, 108, 219, 0, 25, 71, 108, 219, 255, 300, 332], 303: [0, 139, 188, 193, 216, 221, 225, 229, 300, 0, 139, 188, 193, 216, 221, 225, 229, 300, 332], 332: [0, 25, 108, 225, 253, 255, 300, 303, 0, 25, 108, 225, 253, 255, 300, 303], 1: [75, 138, 174, 181, 184, 217, 245, 269, 299, 323], 75: [1, 1, 120, 138, 174, 184, 245, 292, 299, 323, 324], 

In [125]:
# Gilbert 100 nodos
# Parámetros
k = 12  # Constante de repulsión
C = 0.005  # Constante de atracción
t = 0.001  # Factor de tiempo
max_iterations = 100  # Número máximo de iteraciones
Grafo_dibujo(grafo7,k,C,t,max_iterations)

{0: [11, 14, 20, 33, 34, 37, 39, 57, 74, 80, 89, 97, 35, 47, 64, 71], 11: [0, 4, 7, 2, 6, 49, 60, 63, 77, 78, 90, 95, 21, 28, 29, 31, 35, 41, 59, 71, 79, 83, 91, 93], 14: [0, 8, 16, 17, 19, 22, 37, 60, 72, 77, 84, 93, 95, 28, 35, 81, 86], 20: [0, 0, 13, 16, 17, 32, 37, 38, 53, 56, 57, 58, 60, 61, 67, 72, 77, 80, 88, 96, 22, 24, 33, 43, 47, 65, 76, 85, 99], 33: [0, 5, 8, 20, 59, 64, 73, 74, 83, 35, 41, 44, 46, 48, 53, 61, 72, 82, 90, 99], 34: [0, 1, 17, 18, 2, 3, 9, 26, 27, 74, 83, 86, 91, 98, 99, 47, 67, 75, 93, 94], 37: [0, 14, 18, 20, 21, 38, 59, 69, 72, 76, 80, 84, 86, 91, 99, 41, 55, 58, 83, 89, 90, 98], 39: [0, 13, 27, 31, 38, 5, 10, 12, 23, 32, 41, 44, 49, 51, 54, 66, 95, 48, 76, 79], 57: [0, 9, 20, 25, 29, 45, 55, 0, 5, 10, 18, 35, 36, 45, 53, 58, 62, 64, 65, 82, 83, 93, 95, 60, 70], 74: [0, 9, 21, 22, 32, 33, 34, 50, 63, 66, 22, 41, 44, 69, 70, 73, 90, 77, 80, 83, 95], 80: [0, 3, 16, 19, 20, 26, 31, 37, 40, 41, 45, 56, 70, 75, 2, 4, 9, 10, 21, 26, 27, 31, 41, 53, 74, 79, 85, 93

In [127]:
# Gilbert 350 nodos
# Parámetros
k = 5.7  # Constante de repulsión
C = 0.005  # Constante de atracción
t = 0.0007  # Factor de tiempo
max_iterations = 100  # Número máximo de iteraciones
Grafo_dibujo(grafo8,k,C,t,max_iterations)

{0: [110, 111, 265, 205], 110: [0, 17, 107, 114, 287, 331], 111: [0, 41, 67, 150, 191], 265: [0, 16, 64, 250, 61, 218, 220, 252, 313], 1: [218, 291, 70, 175, 211], 218: [1, 56, 155, 223, 269, 261, 265], 291: [1, 272, 80, 114, 241, 248, 314, 341], 2: [58, 208, 317, 340, 89], 58: [2, 38, 84, 183, 202, 301, 320, 330, 337, 125, 333], 208: [2, 163, 204, 94, 144, 196, 272, 315, 333, 342], 317: [2, 17, 211, 221, 270, 51, 89, 200, 240, 322, 344], 340: [2, 59, 69, 105, 239, 268, 41, 147, 186, 227, 321], 3: [80, 130, 136, 203, 65, 75, 186, 193, 198], 80: [3, 47, 40, 94, 198, 263, 271, 291, 293, 294], 130: [3, 8, 9, 108, 127, 55, 134, 143, 267, 297, 309], 136: [3, 82, 116, 126, 147, 154, 175, 195, 295, 305], 203: [3, 142, 53, 70, 117, 188, 316], 4: [24, 33, 84, 95, 118, 230, 170], 24: [4, 101, 176, 188, 251, 308, 81, 118, 173], 33: [4, 38, 41, 301, 134, 281, 338], 84: [4, 58, 72, 4, 106, 146, 177, 191, 211, 299, 313, 328], 95: [4, 156, 185, 205, 125, 175, 216, 329, 342], 118: [4, 24, 179, 216, 23

In [130]:
# Dorogovtsev 100 nodos
# Parámetros
k = 7.7  # Constante de repulsión
C = 0.005  # Constante de atracción
t = 0.003  # Factor de tiempo
max_iterations = 100  # Número máximo de iteraciones
Grafo_dibujo(grafo9,k,C,t,max_iterations)

{0: [4, 7, 9, 19, 34], 1: [2, 3, 11, 12, 16, 20, 22, 25, 39, 41, 50, 53, 60, 75], 2: [1, 3, 4, 5, 8, 15, 23, 40, 64, 72, 83, 96], 3: [2, 1, 8, 9, 13, 15, 21, 28, 31, 48], 4: [2, 0, 5, 6, 20, 38, 42, 47, 55], 5: [2, 4, 7, 8, 11, 12, 18, 20, 21, 23, 24, 30, 44, 79], 6: [4, 7, 8, 13, 22, 29, 42, 58, 70, 95], 7: [5, 6, 0, 15, 18, 19, 29, 41, 65], 8: [3, 6, 2, 5, 10, 24, 26, 27, 32, 53, 82, 95], 9: [3, 0, 10, 11, 12, 13, 15, 16, 28, 45, 55, 68, 69, 76, 80], 10: [8, 9, 12, 14, 17, 40, 57, 59, 88], 11: [1, 9, 5, 20, 24, 28, 49, 51, 88, 99], 12: [9, 1, 10, 5, 17, 19, 23, 31, 32, 41, 58], 13: [6, 9, 3, 14, 18, 32, 42, 80], 14: [10, 13, 16, 17, 25, 28, 33, 35, 39, 53, 60, 78, 87], 15: [2, 3, 7, 9, 22, 36, 55, 70, 72, 75], 16: [1, 14, 9, 17, 18, 21, 50, 66], 17: [12, 16, 14, 10, 22, 29, 33, 37, 49, 52, 58, 60, 66], 18: [5, 7, 13, 16, 19, 25, 27, 35, 37, 38, 54, 65, 67, 85], 19: [18, 7, 12, 0, 41, 43, 45, 50, 77, 93], 20: [4, 5, 1, 11, 26, 29, 32, 37, 52, 63, 72], 21: [3, 16, 5, 27, 31, 42, 79, 93

In [132]:
# Dorogovtsev 350 nodos
# Parámetros
k = 7.7  # Constante de repulsión
C = 0.005  # Constante de atracción
t = 0.001  # Factor de tiempo
max_iterations = 100  # Número máximo de iteraciones
Grafo_dibujo(grafo10,k,C,t,max_iterations)

{0: [5, 6, 9, 10, 12, 14, 16, 21, 22, 23, 29, 30, 50, 128, 132, 133, 143, 161, 321], 1: [2, 3, 7, 12, 13, 15, 27, 30, 36, 41, 59, 61, 131, 152, 210], 2: [1, 3, 4, 11, 13, 17, 32, 34, 37, 38, 44, 50, 55, 59, 68, 96, 124, 126, 160, 270, 298, 314, 346, 348], 3: [2, 1, 4, 5, 7, 8, 10, 11, 15, 21, 25, 36, 41, 91, 94, 104, 109, 165, 235, 323], 4: [3, 2, 5, 7, 8, 12, 18, 31, 50, 56, 59, 63, 75, 80, 83, 84, 115, 179, 206, 229], 5: [3, 0, 4, 6, 8, 16, 22, 28, 46, 49, 51, 62, 127, 146, 210, 297], 6: [5, 0, 8, 15, 32, 42, 44, 45, 47, 55, 91, 172, 186, 299], 7: [4, 1, 3, 9, 43, 66, 77, 121], 8: [6, 3, 5, 4, 19, 25, 26, 37, 40, 72, 74, 115, 238], 9: [7, 0, 12, 14, 17, 18, 20, 32, 40, 42, 54, 167, 176, 191, 198, 264, 309, 348], 10: [3, 0, 11, 15, 17, 30, 35, 47, 116, 233], 11: [2, 10, 3, 18, 28, 29, 47, 49, 69, 76, 87, 139, 171, 221, 255, 265, 295, 297], 12: [9, 4, 1, 0, 13, 22, 24, 25, 35, 43, 60, 79, 84, 93, 100, 118, 147, 166, 193, 224, 242, 293, 297], 13: [1, 12, 2, 23, 29, 33, 82, 88, 95, 265, 

In [138]:
# Barabasi 100 nodos
# Parámetros
k = 12  # Constante de repulsión
C = 0.005  # Constante de atracción
t = 0.001  # Factor de tiempo
max_iterations = 100  # Número máximo de iteraciones
Grafo_dibujo(grafo11,k,C,t,max_iterations)

{1: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 16, 17, 18, 23, 25, 26, 30, 32, 33, 35, 36, 39, 53, 55, 60, 61, 62, 64, 65, 70, 74, 78, 88, 91, 98], 2: [1, 3, 4, 5, 6, 7, 8, 9, 11, 12, 15, 16, 19, 20, 21, 25, 26, 30, 33, 35, 36, 45, 54, 65, 80, 82], 3: [1, 2, 4, 5, 6, 7, 8, 9, 11, 12, 15, 17, 18, 19, 22, 25, 28, 34, 35, 40, 41, 42, 43, 47, 48, 49, 51, 54, 59, 64, 66, 69, 71, 75, 81, 85, 89, 94], 4: [1, 2, 3, 5, 6, 7, 8, 9, 12, 13, 14, 20, 27, 30, 31, 32, 38, 39, 43, 55, 61, 63, 69, 71, 76, 79, 88, 90], 5: [1, 2, 3, 4, 6, 7, 8, 9, 10, 13, 15, 16, 17, 19, 22, 27, 29, 30, 34, 37, 40, 41, 44, 46, 49, 51, 53, 54, 56, 57, 58, 59, 61, 62, 63, 69, 74, 80, 82, 86, 88, 89, 91, 93, 94, 97, 99], 6: [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 14, 17, 20, 21, 22, 25, 26, 32, 35, 36, 37, 38, 43, 44, 46, 47, 51, 60, 62, 63, 64, 76, 87, 92, 95, 97, 99], 7: [1, 2, 3, 4, 5, 6, 8, 9, 10, 13, 14, 15, 18, 20, 23, 24, 26, 27, 28, 29, 31, 36, 41, 43, 52, 55, 56, 57, 59, 60, 62, 65, 70, 71, 73, 75, 82, 91, 97], 8: [1, 2, 3,

In [140]:
# Barabasi 350 nodos
# Parámetros
k = 15  # Constante de repulsión
C = 0.005  # Constante de atracción
t = 0.001  # Factor de tiempo
max_iterations = 100  # Número máximo de iteraciones
Grafo_dibujo(grafo12,k,C,t,max_iterations)

{1: [2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 23, 26, 36, 40, 41, 42, 49, 55, 60, 63, 64, 66, 69, 73, 86, 88, 90, 91, 98, 116, 117, 140, 159, 164, 187, 188, 190, 200, 208, 218, 236, 278, 284, 294, 299, 303, 317, 322, 343], 2: [1, 3, 4, 5, 6, 7, 8, 9, 11, 14, 17, 19, 20, 23, 24, 27, 32, 40, 45, 46, 47, 58, 68, 86, 94, 95, 103, 106, 116, 117, 119, 127, 154, 155, 160, 161, 174, 181, 183, 186, 188, 189, 198, 206, 217, 241, 245, 257, 274, 276, 280, 299, 313, 345, 347], 3: [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 18, 20, 21, 22, 23, 24, 25, 29, 34, 35, 38, 40, 47, 51, 57, 66, 71, 73, 74, 77, 78, 84, 85, 86, 89, 91, 94, 100, 101, 102, 105, 106, 108, 110, 117, 119, 127, 131, 141, 144, 149, 158, 174, 185, 192, 195, 203, 207, 209, 227, 231, 237, 244, 259, 273, 284, 303, 317, 324, 341, 345], 4: [1, 2, 3, 5, 6, 7, 8, 9, 10, 12, 13, 18, 19, 21, 23, 28, 29, 32, 35, 36, 37, 39, 42, 43, 44, 45, 46, 53, 54, 55, 57, 62, 75, 86, 88, 96, 98, 100, 102, 106, 107, 117, 143, 145, 146, 151, 155, 158, 16